<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/NBA_CareerLength_Undrafted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crawling Data: 드래프트X & NBA경력존재 & 은퇴선수

Jupyter notebook

- Stathead.com
- Player Season & Career Stats Finder - Pro Basketball
- Find combined seasons or careers matching criteria
- Season from: 1990/91, to: 2021/22
- Player Debut from: 1990/91 to: 2021/22
- Active: Inactive
- Draft Status: Undrafted

In [ ]:
pip install selenium

In [ ]:
pip install webdriver_manager

In [ ]:
pip install undetected-chromedriver


In [ ]:

#draft list의 선수별 데이터 가져오기(career length)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
import undetected_chromedriver as uc
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.common.exceptions import TimeoutException

# # sheet 생성
# wb = openpyxl.Workbook()
# sheet = wb.active
# sheet.append(["no", "displayName", "team", "officialPosition", "averageSalary"])

class Infos:

    def __init__(self):
        self.name:str = ""
        self.text:list[str] = []
        self.href:list[str] = []

    def get_instagram_link(self):
        for h in self.href:
            if("://instagram.com" in h):
                return h
        return ""

    def get_twitter_link(self):
        for h in self.href:
            if("://twitter.com" in h):
                return h
        return ""

    def get_position(self):
        for t in self.text:
            if("Position:" in t):
                return t.split(":")[1].split("▪")[0].strip()
        return ""

    def get_shoots(self):
        for t in self.text:
            if("Shoots:" in t):
                return t.split(":")[2].strip()
        return ""

    def get_height_inch(self):
        for t in self.text:
            if("lb" in t and "cm" in t and "kg" in t):
                return t.split("(")[0].split(",")[0].strip()
        return ""

    def get_weight_lb(self):
        for t in self.text:
            if("lb" in t and "cm" in t and "kg" in t):
                return t.split("(")[0].replace("lb", "").split(",")[1].strip()
        return ""

    def get_height_cm(self):
        for t in self.text:
            if("lb" in t and "cm" in t and "kg" in t):
                return t.split("(")[1].replace("cm", "").split(",")[0].strip()
        return ""

    def get_weight_kg(self):
        for t in self.text:
            if("lb" in t and "cm" in t and "kg" in t):
                return t.split("(")[1].replace("kg", "").replace("years","").replace("year", "").split(",")[1].strip(")")
        return ""

    def is_died(self):
        for t in self.text:
            if("Died:" in t):
                return True
        return False

    def get_born(self):
        if(self.is_died()):
            for t in self.text:
                if("Born:" in t):
                    return t.split(":")[1].strip()
        else:
            for t in self.text:
                if("Born:" in t):
                    part1 = t.split(":")[1].split("(")[0]
                    part2 = t.split(")")[1]
                    return f"{part1}{part2}".strip()
        return ""

    def get_age(self):
        if(self.is_died()):
            for t in self.text:
                if("Died:" in t):
                    return t.split("(Aged")[1].split(")")[0].strip()
        else:
            for t in self.text:
                if("Born:" in t):
                    return t.split("(")[1].split(")")[0].split(":")[1].strip()
        return ""

    def get_college(self):
        for t in self.text:
            if "College" in t:
                try:
                    return t.split(":")[1].strip()
                except IndexError:
                    return ""
        return ""

    def get_high_school(self):
        for t in self.text:
            if("High School:" in t):
                return t.split(":")[1].strip()
        return ""

    def get_recuiting_rank(self):
        for t in self.text:
            if("Recruiting Rank:" in t):
                return t.split(":")[1].strip()
        return ""

    def get_draft_team(self):
        for t in self.text:
            if("Draft:" in t):
                return t.split(":")[1].split(",")[0].strip()
        return ""

    def get_draft_num(self):
        for t in self.text:
            if("Draft:" in t):
                part1 = t.split(",")[1]
                part2 = t.split(",")[2]
                return f"{part1},{part2}".strip()
        return ""

    def get_draft_year(self):
        for t in self.text:
            if("Draft:" in t):
                return t.split(",")[3].split()[0].strip()
        return ""

    def get_nba_debut(self):
        for t in self.text:
            if("NBA Debut:" in t):
                return t.split(":")[1].strip()
        return ""

    def get_exprtience(self):
        for t in self.text:
            if("Experience:" in t or "Career Length:" in t):
                return t.split(":")[1].strip()
        return ""

    def display(self):
        print(self.name)
        print(self.get_instagram_link())
        print(self.get_twitter_link())
        print(self.get_position())
        print(self.get_shoots())
        print(self.get_height_inch())
        print(self.get_weight_lb())
        print(self.get_height_cm())
        print(self.get_weight_kg())
        print(self.get_born())
        print(self.get_age())
        print(self.get_college())
        print(self.get_high_school())
        print(self.get_recuiting_rank())
        print(self.get_draft_team())
        print(self.get_draft_num())
        print(self.get_draft_year())
        print(self.get_nba_debut())
        print(self.get_exprtience())


def init_driver():
    browser = uc.Chrome()
    browser.get('https://velog.io')
    return browser


def do_login(browser):
    WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="html"]/body/div/div[2]/div[2]/div/header/div/div[2]/button'))).click()
    time.sleep(3)  # 로그인 버튼을 클릭하고 충분한 시간을 기다립니다.
    print("Please click the Google login button manually.")
    input("Press Enter to continue after logging in...")
    # 사용자가 구글로그인 버튼을 클릭하고 Enter 키를 누르면 코드가 다음 단계로 진행됩니다.
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
    email_input = browser.find_element(By.XPATH, '//*[@id="identifierId"]')
    email_input.send_keys('mediwons2024@gmail.com')
    browser.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button').click()
    time.sleep(5)
    password_input = browser.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
    password_input.send_keys('apeldnjstm2024')
    browser.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button').click()
    time.sleep(5)

if __name__ == "__main__":
    browser = init_driver()
    do_login(browser)

    # stathead.com으로 이동
    url = 'https://stathead.com/basketball/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=year_id&match=player_season_combined&season_start=1&year_max=2022&debut_yr_end=2022&display_type=totals&order_by_asc=1&season_end=-1&is_active=N&debut_yr_start=1991&draft_status=undrafted&comp_id=NBA&year_min=1991&offset='
    browser.get(url)

    # 두 번째 로그인을 위한 시간을 줍니다.
    print("Please perform the second login manually on stathead.com. Once logged in, press Enter to continue.")
    input("Press Enter to continue...")
    #구글 로그인을 완료하고 Enter키 누르기.

    # 'url' 변수로 할당된 페이지로 이동합니다.
    browser.get(url)
    loop_count = 0

    # Load the page
    browser.get(f'{url}{loop_count * 200}')
    # target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
    # target_element_login.click()
    # target_element_login2 = browser.find_element(By.XPATH, '//*[@id="sh-login-google"]')
    # target_element_login2.click()
    # browser.get(f'{url}{loop_count * 100}')

    is_complete_login = False

    data_dict = {}
    data_dict['Name'] = []
    data_dict['Twitter'] = []
    data_dict['Instagram'] = []
    data_dict['Position'] = []
    data_dict['Shoots'] = []
    data_dict['Height_inch'] = []
    data_dict['Weight_lb'] = []
    data_dict['Height_cm'] = []
    data_dict['Weight_kg'] = []
    data_dict['Born'] = []
    data_dict['Age'] = []
    data_dict['College'] = []
    data_dict['High School'] = []
    data_dict['Recruiting Rank'] = []
    data_dict['Draft_team'] = []
    data_dict['Draft_num'] = []
    data_dict['Draft_year'] = []
    data_dict['NBA Debut'] = []
    data_dict['Experience'] = []

    while True:
        has_next = True

        try:
            next_button = browser.find_element(By.CLASS_NAME, 'next')
        except:
            has_next = False

        print("loop")
        loop_count += 1
        print(loop_count)

        WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.ID, 'stats')))  # Wait for the table to load

        # Extract the data from the table
        table = browser.find_element(By.XPATH, '//*[@id="stats"]')
        trs = table.find_elements(By.XPATH, './/tr')

#         rows = table.find_elements(By.XPATH, './/tr')[0:]  # Exclude header row

        link_list = []

        for tr in trs:
            try:
                url_cell = tr.find_elements(By.XPATH, './/td')[0]
                print(url_cell.text)
                player_url = url_cell.find_element(By.TAG_NAME, 'a').get_attribute('href')
                link_list.append(player_url.strip())


            except Exception as e:
                continue

        # if(is_complete_login == False):
        #     new_login_url = 'https://www.basketball-reference.com/'
        #     browser.get(new_login_url)
        #     time.sleep(3)
        #     target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
        #     target_element_login.click()
        #     time.sleep(3)
        #     is_complete_login = True

        for link in link_list:
            print(link)
            browser.get(link)
            time.sleep(3)
            try:
                more_bio_button = browser.find_element(By.XPATH, '//*[@id="meta_more_button"]')
                more_bio_button.click()
            except Exception as e:
                pass  # Continue without clicking if "More Bio" button is not found
            # Extract the data from the player page
#             data_element = browser.find_element(By.CLASS_NAME, 'stats_pullout')
            data_element = None

            meta = browser.find_element(By.ID, "meta")

            data_div = None
            try:
                picture = meta.find_element(By.CLASS_NAME, "media-item")
                data_div = meta.find_elements(By.TAG_NAME, "div")[1]
            except:
                data_div = meta.find_elements(By.TAG_NAME, "div")[0]

            name = data_div.find_element(By.TAG_NAME, "h1")
            p_tag_list = data_div.find_elements(By.TAG_NAME, "p")
            a_tag_list = data_div.find_elements(By.TAG_NAME, 'a')

            infos = Infos()
            infos.name = name.text

            for p in p_tag_list:
                infos.text.append(p.text)

            for a in a_tag_list:
                infos.href.append(a.get_attribute('href'))

            #infos.display()

            data_dict['Name'].append(infos.name)
            data_dict['Twitter'].append(infos.get_twitter_link())
            data_dict['Instagram'].append(infos.get_instagram_link())
            data_dict['Position'].append(infos.get_position())
            data_dict['Shoots'].append(infos.get_shoots())
            data_dict['Height_inch'].append(infos.get_height_inch())
            data_dict['Weight_lb'].append(infos.get_weight_lb())
            data_dict['Height_cm'].append(infos.get_height_cm())
            data_dict['Weight_kg'].append(infos.get_weight_kg())
            data_dict['Born'].append(infos.get_born())
            data_dict['Age'].append(infos.get_age())
            data_dict['College'].append(infos.get_college())
            data_dict['High School'].append(infos.get_high_school())
            data_dict['Recruiting Rank'].append(infos.get_recuiting_rank())
            data_dict['Draft_team'].append(infos.get_draft_team())
            data_dict['Draft_num'].append(infos.get_draft_num())
            data_dict['Draft_year'].append(infos.get_draft_year())
            data_dict['NBA Debut'].append(infos.get_nba_debut())
            data_dict['Experience'].append(infos.get_exprtience())

        if(has_next):
            browser.get(f'{url}{loop_count * 200}')
            time.sleep(3)
        else:
            break

        try:
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, 'stats')))
        except TimeoutException:
            break


    df = pd.DataFrame(data_dict)
        # 데이터프레임을 CSV 파일로 저장
    df.to_csv('NBA_Player_Undrafted_CareerLength.csv', index=False)

    # Step 4: Process is complete
    print("Crawling complete!")

    # Quit the WebDriver
#     browser.quit()